In [1]:
# BLANK
import torch
from torch import cuda

import os
import shutil
import sys
from pathlib import Path

from tqdm.auto import tqdm, trange
from importlib import reload
from ultralytics import YOLO

# Clone georip repo here: https://www.github.com/joeletho/georip.git

# Cloned repo directory
sys.path.append("path/to/georip")

import georip


In [2]:
# BLANK
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]


In [3]:
# BLANK
print(reload(georip))

<module 'georip.georip' from '/home/joel/Dev/python/georip/georip.py'>
<module 'georip.utils' from '/home/joel/Dev/python/georip/utils.py'>
<module 'georip.modeling' from '/home/joel/Dev/python/georip/modeling/__init__.py'>
<module 'georip.modeling.yolo' from '/home/joel/Dev/python/georip/modeling/yolo.py'>


In [4]:
# BLANK
has_gpu = cuda.is_available()

device = torch.device('cuda' if has_gpu else 'cpu')
print(device)
if has_gpu:
    print(cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 3050 Ti Laptop GPU



#### Example directory structure:
```
Root
  ├── GEORIP_YOLO
  |         ├── datasets
  |         ├── models
  ├── NDVI
  ├── QGIS
  ├── Readme.txt
  ├── Shapefiles
  ...
```

In [5]:
# BLANK
path_map = {}

# Change this to your project path
path_map['ROOT'] = Path("path/to/project/root")

path_map['RESULTS'] = path_map['ROOT'] / 'results'
path_map['PROJECT_NAME'] = 'GEORIP_YOLO'
path_map['georip'] = path_map['ROOT'] / path_map["PROJECT_NAME"]
path_map['GEORIP_DS'] = path_map['georip'] / 'datasets'
path_map['GEORIP_MODELS'] = path_map['georip'] / 'models'

In [6]:
# BLANK
def make_directories(paths_map, verbose=True, exist_ok=False):
    if verbose:
        print("Creating directory structure")
    for name, path in paths_map.items():
        if isinstance(path, Path):
            if path.is_file() or len(path.suffix) > 0:
                paths_map[name] = path.resolve()
            else:
                path = path.resolve()
                paths_map[name] = path
                path.mkdir(parents=True, exist_ok=exist_ok)
                if verbose:
                    print('  ',path)
    if verbose:
        print("Complete")

def make_project_paths(root,*, verbose=True, exist_ok=False):    
    paths = {'NDVI': Path(root, 'NDVI', 'NDVI Difference Rasters')}
    paths['SHAPE_FILES'] = Path(root, 'Shapefiles')

    paths['GEORIP_DS_META'] = path_map['georip_DS'] / 'meta'
    paths['GEORIP_DS_CSV'] = paths['georip_DS_META'] / 'csv'
    paths['GEORIP_DS_SHP'] = paths['georip_DS_META'] / 'shp'
    
    # Data
    paths['PRED_SHP'] = paths['SHAPE_FILES'] / 'ModelPredictions'
    paths['SHPZ10_SHP'] = paths['SHAPE_FILES'] / 'Treatments_UTMz10_Only_08-18-24' / 'Treatments_UTMz10_Only_08-18-24.shp'
    paths['SHPZ11_SHP'] = paths['SHAPE_FILES'] / 'Treatments_UTMz11_Only_08-18-24' / 'Treatments_UTMz11_Only_08-18-24.shp'
    for name, path in paths.items():
        path_map[name] = path
    make_directories(paths, verbose=verbose, exist_ok=exist_ok)

        
def make_dataset_paths(ds_root, models_root, model_name, *,verbose=True, exist_ok=False):
    ds_root = Path(ds_root)
    models_root = Path(models_root)
    paths = {}
        
    paths['MODEL_NAME'] = model_name
    paths['GEORIP_MODEL'] = models_root / paths['MODEL_NAME']
    paths['GEORIP_DS_MODEL'] = ds_root / paths['MODEL_NAME']
    paths['GEORIP_DS_MODEL_META'] = paths['georip_DS_MODEL'] / 'meta'
    paths['GEORIP_DS_MODEL_SHP'] = paths['georip_DS_MODEL_META'] / 'shp'
    paths['GEORIP_DS_MODEL_CSV'] = paths['georip_DS_MODEL_META'] / 'csv'
    
    paths['GEORIP_DS_DATA'] = paths['georip_DS_MODEL'] / 'meta'
    paths['GEORIP_DS_CONFIG_FILE'] = paths['georip_DS_MODEL'] / 'config' / 'data.yaml'
    paths['GEORIP_DS_YOLO_DATA_FILE'] = paths['georip_DS_DATA'] / 'yolo_ndvi_ds.csv'
    
    # Images and labels
    paths['GEORIP_DS_IMAGES'] = paths['georip_DS_MODEL'] / 'images'
    paths['GEORIP_DS_LABELS'] = paths['georip_DS_MODEL'] / 'labels'
    paths['GEORIP_DS_LABELS_GENERATED'] = paths['georip_DS_LABELS'] / 'generated'
    
    paths['GEORIP_DS_CHIPS'] = paths["georip_DS_IMAGES"] / 'chips'
    paths['GEORIP_DS_PNGS'] = paths["georip_DS_IMAGES"] / 'png'
    paths['GEORIP_DS_TIFS'] = paths["georip_DS_IMAGES"] / 'tif'
    
    paths['GEORIP_DS_IMAGES_TRAIN'] = paths['georip_DS_IMAGES'] / 'train'
    paths['GEORIP_DS_IMAGES_TEST'] = paths['georip_DS_IMAGES'] / 'test'
    paths['GEORIP_DS_IMAGES_VAL'] = paths['georip_DS_IMAGES'] / 'val'
    
    paths['GEORIP_DS_LABELS_TRAIN'] = paths['georip_DS_LABELS'] / 'train'
    paths['GEORIP_DS_LABELS_TEST'] = paths['georip_DS_LABELS'] / 'test'
    paths['GEORIP_DS_LABELS_VAL'] = paths['georip_DS_LABELS'] / 'val'

    # Metadata

    # Zone 10
    paths['CSVZ10'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz10.csv'
    paths['CSVZ10_NORM'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz10_normalized.csv'
    paths['CSVZ10_CLEANED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz10_normalized_cleaned.csv'
    paths['CSVZ10_CHIPPED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz10_normalized_chipped.csv'
    paths['CSVZ10_CHIP_LABELS_UTM'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z10utm_chip_labels.csv'
    paths['CSVZ10_CHIP_LABELS_PIXEL'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z10pixel_chip_labels.csv'
    paths['CSVZ10_CHIP_LABELS_PIXEL_ENCODED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z10pixel_chip_labels_encoded.csv'
    paths['CSVZ10_CHIP_LABELS_PREYOLO'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z10pixel_chip_labels_encoded_preyolo.csv'
    
    # Zone 11
    paths['CSVZ11'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz11.csv'
    paths['CSVZ11_NORM'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz11_normalized.csv'
    paths['CSVZ11_CLEANED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz11_normalized_cleaned.csv'
    paths['CSVZ11_CHIPPED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_UTMz11_normalized_chipped.csv'
    paths['CSVZ11_CHIP_LABELS_UTM'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z11utm_chip_labels.csv'
    paths['CSVZ11_CHIP_LABELS_PIXEL'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z11pixel_chip_labels.csv'
    paths['CSVZ11_CHIP_LABELS_PIXEL_ENCODED'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z11pixel_chip_labels_encoded.csv'
    paths['CSVZ11_CHIP_LABELS_PREYOLO'] = paths['GEORIP_DS_MODEL_CSV'] / 'Treatments_z11pixel_chip_labels_encoded_preyolo.csv'

    for name, path in paths.items():
        path_map[name] = path
    
    make_directories(paths, verbose=verbose, exist_ok=exist_ok)

    path_map['SHPZ10_PRED_SHP'] = path_map['PRED_SHP'] / f"Treatmentsz10_{paths['MODEL_NAME']}.shp"
    path_map['SHPZ11_PRED_SHP'] = path_map['PRED_SHP'] / f"Treatmentsz11_{paths['MODEL_NAME']}.shp"

make_project_paths(path_map['ROOT'], exist_ok=True)

Creating directory structure
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/NDVI/NDVI Difference Rasters
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/Shapefiles
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/GEORIP_YOLO/datasets/meta
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/GEORIP_YOLO/datasets/meta/csv
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/GEORIP_YOLO/datasets/meta/shp
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/Shapefiles/ModelPredictions
Complete


In [ ]:
# BLANK
CONF=0.9

In [7]:
# BLANK
print(path_map['RESULTS'])

dirs = []
for path in path_map['RESULTS'].iterdir():
    if path.is_file():
        continue
    dirs.append(path)

n_dirs = 0
for dir in dirs:
    n_dirs += len(os.listdir(dir))
    
pbar = trange(n_dirs, desc='Predicting on model results')

for dir in dirs:
    for res_dir in dir.iterdir():
        if res_dir.is_file():
            pbar.update()
            continue
            
        model_name = res_dir.name
        years = None
        parts = model_name.split('_')
        for part in parts:
            if part.startswith('years'):
                years = part.split('=')[1]
        if years is None:
            raise ValueError(f"Error parsing 'years' in '{model_name}'")
    
        args_path = res_dir / 'train' / 'args.yaml'
        if not args_path.exists():
            raise FileNotFoundError(args_path)
    
        imgsz = None
        batchsz = None
        with open(args_path) as f:
            for line in f:
                parts = line.strip('\n').split(':')
                parts[0] = parts[0].strip()
                parts[1] = parts[1].strip()
                if parts[0] == 'imgsz':
                    imgsz = int(parts[1])
                elif parts[0] == 'batch':
                    batchsz= max(1, int(parts[1])-2)
        if imgsz is None:
            raise ValueError(f"Error parsing image size in '{args_path}'")
        if batchsz is None:
            raise ValueError(f"Error parsing batch size in '{args_path}'")
    
        model_path = res_dir / 'train' / 'weights' / 'best.pt'
        if not model_path.exists():
            raise FileNotFoundError(model_path)
    
        model = YOLO(model_path)
        model.to('cuda')
        
        imgs = [img for img in georip.io.collect_files_with_suffix(".tif", path_map['NDVI']) if years in str(img)]
        results, gdfs = georip.modeling.yolo.predict_geotiffs(
            model,
            imgs,
            confidence=CONF, 
            chip_size=(imgsz, imgsz), 
            imgsz=imgsz,
            device=0,
            max_images=1,
            batch_size=batchsz,
        )
        print(f"Processed {model_name}. Saving...", end=' ')
        for i, gdf in enumerate(gdfs):
            georip.io.save_as_shp(gdf, path_map['PRED_SHP'] / model_name / f'TreamentPredictions_{i}_conf={CONF}.shp', exist_ok=True)
        print(f"saved {len(gdf)} files")
        pbar.update()
        
pbar.set_description("Complete")
pbar.close()

In [15]:
# 